In [1]:
import pickle
import cv2
import mediapipe as mp
import numpy as np

In [2]:
# Load model
model_dict = pickle.load(open('./model.p', 'rb'))
model = model_dict['model']

In [3]:
# Initialize video capture and MediaPipe Hands
cap = cv2.VideoCapture(0)
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
hands = mp_hands.Hands(static_image_mode=False, min_detection_confidence=0.3)

In [1]:
# import cv2
# import numpy as np
# import mediapipe as mp

# # Label dictionary for character predictions
# labels_dict = {0: 'අ', 1: 'ආ', 2: 'ඇ'}

# # Initialize video capture
# cap = cv2.VideoCapture(0)

# # Initialize Mediapipe Hands
# mp_hands = mp.solutions.hands
# hands = mp_hands.Hands()
# mp_drawing = mp.solutions.drawing_utils

# # Assuming `model` is already trained and loaded
# while True:
#     data_aux = []
#     x_ = []
#     y_ = []

#     ret, frame = cap.read()
#     if not ret:
#         break

#     H, W, _ = frame.shape
#     frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     results = hands.process(frame_rgb)

#     if results.multi_hand_landmarks:
#         for hand_landmarks in results.multi_hand_landmarks:
#             mp_drawing.draw_landmarks(
#                 frame,
#                 hand_landmarks,
#                 mp_hands.HAND_CONNECTIONS,
#                 mp_drawing_styles.get_default_hand_landmarks_style(),
#                 mp_drawing_styles.get_default_hand_connections_style()
#             )

#             for landmark in hand_landmarks.landmark:
#                 x_.append(landmark.x)
#                 y_.append(landmark.y)

#             # Use only the first 21 landmarks (assuming 21 is half of 42 features)
#             for i in range(21):  # 21 landmarks for 42 features (x and y)
#                 data_aux.append((x_[i] - min(x_)) / (max(x_) - min(x_)))
#                 data_aux.append((y_[i] - min(y_)) / (max(y_) - min(y_)))

#         x1 = max(0, int(min(x_) * W) - 10)
#         y1 = max(0, int(min(y_) * H) - 10)
#         x2 = min(W, int(max(x_) * W) + 10)
#         y2 = min(H, int(max(y_) * H) + 10)

#         # Check if we have the right number of features
#         print(f"Feature count: {len(data_aux)}")

#         if len(data_aux) == 42:  # Ensure exactly 42 features
#             data_aux = np.asarray(data_aux).reshape(1, -1)
#             prediction = model.predict(data_aux)
#             predicted_character = labels_dict[int(prediction[0])]

#             cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
#             cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3, cv2.LINE_AA)
#         else:
#             print("Not enough features collected.")

#     cv2.imshow('frame', frame)

#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()


Model loaded successfully.
Feature capture started.
Feature count: 42


ValueError: Unrecognized data type: x=[0.24120488800549053, 1.0, 0.6937507124705689, 0.8990371985032555, 0.9057369185712403, 0.7056790022649976, 0.9467768652420852, 0.525807541471704, 1.0, 0.39026506625768903, 0.6736694802706051, 0.49409920192560297, 0.7824388143176209, 0.30520251256162473, 0.8302748506216027, 0.18640122764291142, 0.8517760982367321, 0.08245851646205951, 0.4501800703393166, 0.4683188782936545, 0.5275061723993476, 0.25533928220639646, 0.5631709830850691, 0.11651409884099338, 0.5807576887009087, 0.0, 0.22954898677889182, 0.47779349706639024, 0.2837783762660866, 0.2709084782387454, 0.34145061118833286, 0.1424759240456597, 0.3891468507979905, 0.034684551154154745, 0.0, 0.5197089702899528, 0.013250193393245756, 0.36608171978878956, 0.03225714898246302, 0.2606386251914124, 0.05730163294735998, 0.16211389127538064] (of type <class 'list'>)

In [1]:
import cv2
import numpy as np
import mediapipe as mp
import pickle  # For loading the model

# Label dictionary for character predictions
labels_dict = {0: 'අ', 1: 'ආ', 2: 'ඇ'}

# Load the trained model (ensure the model is trained and saved properly)
try:
    with open('model.p', 'rb') as f:
        model = pickle.load(f)['model']
        print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    model = None

# Initialize video capture
cap = cv2.VideoCapture(0)

# Initialize Mediapipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

capture_started = False  # Flag to start capturing features

while True:
    data_aux = []
    x_ = []
    y_ = []

    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame.")
        break

    H, W, _ = frame.shape
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    # Display message to user
    if not capture_started:
        cv2.putText(frame, "Press 's' to start capturing hand features", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # If capture has started, detect hand landmarks
    if capture_started:
        # If hands are detected
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks on the frame
                mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS
                )

                # Collect x and y coordinates for the landmarks
                for landmark in hand_landmarks.landmark:
                    x_.append(landmark.x)
                    y_.append(landmark.y)

                # Use only the first 21 landmarks (each having x and y coordinates) for 42 features
                for i in range(21):  # Only 21 landmarks (each having x and y)
                    data_aux.append((x_[i] - min(x_)) / (max(x_) - min(x_)))
                    data_aux.append((y_[i] - min(y_)) / (max(y_) - min(y_)))

            # Ensure exactly 42 features
            print(f"Feature count: {len(data_aux)}")

            # Convert the list to a NumPy array and reshape to match the model's input
            data_aux = np.array(data_aux).reshape(1, -1)  # Reshape to (1, 42)

            # Ensure that model is loaded before making predictions
            if model:
                prediction = model.predict(data_aux)
                
                # Convert the prediction to a scalar and map it to the corresponding character
                predicted_character = labels_dict[int(prediction.item())]

                # Draw the bounding box around the detected hand
                x1 = max(0, int(min(x_) * W) - 10)
                y1 = max(0, int(min(y_) * H) - 10)
                x2 = min(W, int(max(x_) * W) + 10)
                y2 = min(H, int(max(y_) * H) + 10)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
                cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3, cv2.LINE_AA)
            else:
                print("Model not loaded. Cannot make prediction.")
    
    # Display the frame
    cv2.imshow('frame', frame)

    # Wait for key press to either start capture or exit
    key = cv2.waitKey(1) & 0xFF
    if key == ord('s'):  # Start capturing features when 's' is pressed
        capture_started = True
        print("Feature capture started.")
    elif key == ord('q'):  # Quit the loop when 'q' is pressed
        break

cap.release()
cv2.destroyAllWindows()


Model loaded successfully.
Feature capture started.
Feature count: 42
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step


ValueError: can only convert an array of size 1 to a Python scalar

In [1]:
import cv2
import numpy as np
import mediapipe as mp
import pickle  # For loading the model

# Label dictionary for character predictions
labels_dict = {0: 'අ', 1: 'ආ', 2: 'ඇ'}

# Load the trained model (ensure the model is trained and saved properly)
try:
    with open('model.p', 'rb') as f:
        model = pickle.load(f)['model']
        print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    model = None

# Initialize video capture
cap = cv2.VideoCapture(0)

# Initialize Mediapipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

capture_started = False  # Flag to start capturing features

while True:
    data_aux = []
    x_ = []
    y_ = []

    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame.")
        break

    H, W, _ = frame.shape
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    # Display message to user
    if not capture_started:
        cv2.putText(frame, "Press 's' to start capturing hand features", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # If capture has started, detect hand landmarks
    if capture_started:
        # If hands are detected
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Draw hand landmarks on the frame
                mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS
                )

                # Collect x and y coordinates for the landmarks
                for landmark in hand_landmarks.landmark:
                    x_.append(landmark.x)
                    y_.append(landmark.y)

                # Use only the first 21 landmarks (each having x and y coordinates) for 42 features
                for i in range(21):  # Only 21 landmarks (each having x and y)
                    data_aux.append((x_[i] - min(x_)) / (max(x_) - min(x_)))
                    data_aux.append((y_[i] - min(y_)) / (max(y_) - min(y_)))

            # Ensure exactly 42 features
            print(f"Feature count: {len(data_aux)}")

            # Convert the list to a NumPy array and reshape to match the model's input
            data_aux = np.array(data_aux).reshape(1, -1)  # Reshape to (1, 42)

            # Ensure that model is loaded before making predictions
            if model:
                prediction = model.predict(data_aux)

                # Access the first prediction value if it's a 1D array
                predicted_class = int(np.argmax(prediction, axis=-1))  # Get the index of the max value if it's a multi-class output
                predicted_character = labels_dict[predicted_class]

                # Draw the bounding box around the detected hand
                x1 = max(0, int(min(x_) * W) - 10)
                y1 = max(0, int(min(y_) * H) - 10)
                x2 = min(W, int(max(x_) * W) + 10)
                y2 = min(H, int(max(y_) * H) + 10)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
                cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3, cv2.LINE_AA)
            else:
                print("Model not loaded. Cannot make prediction.")
    
    # Display the frame
    cv2.imshow('frame', frame)

    # Wait for key press to either start capture or exit
    key = cv2.waitKey(1) & 0xFF
    if key == ord('s'):  # Start capturing features when 's' is pressed
        capture_started = True
        print("Feature capture started.")
    elif key == ord('q'):  # Quit the loop when 'q' is pressed
        break

cap.release()
cv2.destroyAllWindows()


Model loaded successfully.
Feature capture started.
Feature count: 42
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 325ms/step
Feature count: 42
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step

C:\Users\acer\AppData\Local\Temp\ipykernel_14812\2052500042.py:79: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_class = int(np.argmax(prediction, axis=-1))  # Get the index of the max value if it's a multi-class output


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Feature count: 42
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Feature count: 42
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Feature count: 42
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Feature count: 42
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Feature count: 42
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Feature count: 42
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Feature count: 42
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Feature count: 42
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Feature count: 42
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Feature count: 42
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Feature count: 42
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Feature count: 42
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Feature count: 42
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Feature count: 42
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Feature count: 42
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Feature count: 42
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Feature count: 42
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Feature co

InvalidArgumentError: Graph execution error:

Detected at node sequential_1/dense_1/Relu defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\acer\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\acer\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance

  File "C:\Users\acer\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start

  File "C:\Users\acer\anaconda3\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "C:\Users\acer\anaconda3\Lib\asyncio\windows_events.py", line 321, in run_forever

  File "C:\Users\acer\anaconda3\Lib\asyncio\base_events.py", line 607, in run_forever

  File "C:\Users\acer\anaconda3\Lib\asyncio\base_events.py", line 1922, in _run_once

  File "C:\Users\acer\anaconda3\Lib\asyncio\events.py", line 80, in _run

  File "C:\Users\acer\anac